In [1]:
import requests
import json
from openai import OpenAI

In [2]:
OLLAMA_ENDPOINT = "http://localhost:11434/v1"
MODEL_NAME = "qwen3:4b"

In [3]:
client = OpenAI(
    base_url=OLLAMA_ENDPOINT,
    api_key="ollama"
)

In [ ]:
def ask_ollama(query):
    
    response = client.chat.completions.create(
        model="qwen3:4b",
        messages=[{"role": "user", "content": query}]
    )

    return response.choices[0].message.content

print(ask_ollama("what is the weather in new orleans today?"))

In [ ]:
# api call to get location data
!curl "https://geocoding-api.open-meteo.com/v1/search?name=new%20orleans&count=1&language=en&format=json"

In [ ]:
# api call to get weather data
!curl "https://api.open-meteo.com/v1/forecast?latitude={29.95465}&longitude={-90.07507}&current_weather=true"

In [17]:
def get_coordinates(city_name):
    """Get latitude and longitude for a city"""
    try: 
        encoded_city = city_name.replace(' ', '%20')
        url = f"https://geocoding-api.open-meteo.com/v1/search?name={encoded_city}&count=1&language=en&format=json"
        response = requests.get(url)
        data = response.json()
        if not data.get('results'):
            return None, None, None
        result = data['results'][0]
        return (result['latitude'], result['longitude'], result['name'])
    except Exception as e:
        print(f"Error getting coordinates: {e}")
        return None, None, None
    
def get_weather(latitude, longitude):
    """Get weather information for a city"""
    try:
        url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"
        response = requests.get(url)
        data = response.json()
        return data
    except Exception as e:
        print(f"Error getting weather: {e}")
        return None

In [29]:
SYSTEM_PROMPT1 = """
You are a heplful AI assitant.
You have access to the following functions to assist the user.

You SHOULD NOT include any other text in your response if you are calling a function.
If you have enough information to answer the user's request or the request does not require a function call, respond directly to the user in plain text.

Here are the available functions:
[
  {
    "name": "get_coordinates",
    "description": "Get latitude and longitude for a city.",
    "parameters": {
      "type": "object",
      "properties": {
        "city_name": {
          "type": "string",
          "description": "The name of the city for which to get coordinates."
        }
      },
      "required": [
        "city_name"
      ]
    }
  },
  {
    "name": "get_weather",
    "description": "Get current weather information for a given latitude and longitude.",
    "parameters": {
      "type": "object",
      "properties": {
        "latitude": {
          "type": "number",
          "description": "The latitude of the location."
        },
        "longitude": {
          "type": "number",
          "description": "The longitude of the location."
        }
      },
      "required": [
        "latitude",
        "longitude"
      ]
    }
  }
]

If you decide to invoke one or more functions, you MUST output a LIST  of objects, where each object is in the format:
{"name": "function_name", "parameters": {"argument_name1": "value1", "argument_name2": "value2"}}

For example:
To make a single tool call you should output: [{"name": "tool_A", "parameters": {"arg": "value"}}]
To make multiple tool calls you should output: [{"name": "tool_A", "parameters": {"arg": "value"}}, {"name": "tool_B", "parameters": {"arg": "other_value"}}]

Remember: 
- Once you make a tool call, the response will be fed back to you.
- Once you do not need to make any more tool calls - you can output the final answer.
- Plan the sequence of your tool calls appropriately.

"""

In [30]:
import re
from pydantic import BaseModel
class Response(BaseModel):
    """Response schema for the thinking models"""
    thoughts: str
    answer: str

def make_messages(query, system_prompt=SYSTEM_PROMPT1):
    """Make messages for the OpenAI API"""
    
    return [{"role": "system", "content": system_prompt},
            {"role": "user", "content": query}]
    
    
def split_thoughts_and_answer(text):
    # Extract thoughts from within <think>...</think>
    thoughts_match = re.search(r"<think>\n?(.*?)</think>", text, re.DOTALL)
    thoughts = thoughts_match.group(1).strip() if thoughts_match else ""

    # Extract the answer (everything after </think>)
    answer_start = text.find("</think>")
    answer = text[answer_start + len("</think>"):].strip()
    
    return {"thoughts": thoughts, "answer": answer}


def ask_ollama_with_function_calling(messages):
    """Ask the OpenAI API with function calling"""
    response = client.chat.completions.create(
        model="qwen3:4b",
        messages=messages,
    )
    content = response.choices[0].message.content
    
    parsed_response = split_thoughts_and_answer(content)
    
    return Response(**parsed_response)


In [ ]:
resp = ask_ollama_with_function_calling(make_messages("what is the weather in new york?"))
print("thoughts: ", resp.thoughts)
print("-"*100)
print("answer: ", resp.answer)

In [21]:
# Helper functions

# Check if the response is a valid tool call
def is_response_valid_tool_call(response):
    """Check if the response is a valid tool call and return the parsed list, or False."""
    # Strip code block markers if present
    if response.startswith("```json") and response.endswith("```"):
        response = response[len("```json"): -len("```")].strip()
    
    # Try to parse the response as JSON
    try:
        tool_calls = json.loads(response)
        if isinstance(tool_calls, list) and all(
            isinstance(call, dict) and 'name' in call and 'parameters' in call for call in tool_calls
        ):
            return tool_calls
        else:
            return False
    except Exception:
        return False

# Create a tool response to be appended to the messages
def create_tool_response(function_name, parameters, function_response):
    """Create a tool response"""
    return f"Function {function_name} called with parameters {parameters} and returned {function_response}"


In [31]:
MAX_TURNS = 5
AVAILABLE_FUNCTIONS = {
    "get_coordinates": get_coordinates,
    "get_weather": get_weather
}
def simple_loop(query, max_turns = MAX_TURNS):
    messages = make_messages(query, SYSTEM_PROMPT1)
    for i in range(max_turns):
        completion_response = ask_ollama_with_function_calling(messages)
        print("-"*100)
        print("Turn: ", i+1)
        print("thoughts: ", completion_response.thoughts)
        print("answer: ", completion_response.answer)
        response = completion_response.answer
        tool_calls_requested = is_response_valid_tool_call(response)
        if not tool_calls_requested:
            return response
        else:
            print(f"Tool calls requested in turn {i+1}:")
            messages.append({"role": "assistant", "content": response})
            for tool_call in tool_calls_requested:
                function_name = tool_call.get("name")
                parameters = tool_call.get("parameters", {})
                print(f" Function name: {function_name}")
                print(f" Parameters: {parameters}")
                if function_name in AVAILABLE_FUNCTIONS:
                    function_to_call = AVAILABLE_FUNCTIONS[function_name]
                    function_response = function_to_call(**parameters)
                    messages.append({"role": "user", "content": create_tool_response(function_name, parameters, function_response)})
                else:
                    raise ValueError(f"Function {function_name} not found")
        

In [ ]:
simple_loop("what is the weather in visakhapatnam?")

In [ ]:
simple_loop("what is the weather in new york?")